# Project: Data Warehouse Implementation & Analytics in Microsoft Fabric

## 1. Project Overview

As our business grows, we need a structured way to store, process, and analyze sales data. Currently, we lack a centralized data solution, making it difficult to track key business metrics, optimize performance, and generate insightful reports.

We need a **data warehouse** that integrates well with **Microsoft Fabric** and allows us to:
- Ingest sales data from external sources.
- Organize and structure the data efficiently.
- Ensure fast and accurate reporting on sales performance.
- Provide clear visibility into customer behavior, product performance, and revenue trends.

## 2. Project Implementation Summary

### 2.1 Data Lakehouse Creation

1. **Create a new workspace**: I created a new workspace named `sales_ws` to group the data warehouse, lakehouse, and other components related to our project.
2. **Create a lakehouse**: I created a lakehouse named `sales_dlh` to store our `sales.csv` file and the staging table `sales_stg`.
3. **Upload data**: I uploaded the `sales.csv` file to the lakehouse and created a staging table (`sales_stg`) that contains the raw data from the `sales.csv` file.

### 2.2 Data Warehouse Creation

1. **Create an empty data warehouse**: Before building our data warehouse, I created an empty one in the same workspace named `sales_dwh`.
2. **Create a schema**: For better organization, I created a schema named `sales_schema` to group the tables related to sales.
3. **Create dimension and fact tables**: I created the dimension and fact tables and loaded data into them.
4. **Conduct analytical analysis**: Finally, I conducted some analytical queries and demonstrated the functionalities of the Fabric notebook in data visualization.


## 3. Implementation

1. **<u>Schema Creation**

In [3]:
%%sql
-- To organize our data warehouse we will create a dedicated schema 'sales_schema' that will group the tables.

if not exists( select * from sys.schemas where name = 'sales_schema')
begin
exec('create schema sales_schema') -- for creating a schema after using the if not exists check, we need to pass by exec function
end

In [4]:
%%sql
-- Checking available tables in sales_dwh data warehouse 
SELECT *
FROM [sales_dwh].[INFORMATION_SCHEMA].[TABLES]

<img src="table 1.png" width=800>

2. **<u>Tables Creation**

In [5]:
%%sql
-- Next I wil create the Dimensional and fact tables to organize our data for analytics.

-- Create Dim_Customer Table
if not exists (select * from INFORMATION_SCHEMA.TABLES where table_schema ='sales_schema' and table_name = 'Dim_Customer')
CREATE TABLE sales_schema.Dim_Customer (
    CustomerID varchar(255) not null
    ,CustomerName varchar(255)
    ,EmailAddress varchar(255) not null
    --,Constraint PK_Dim_Customer primary key (CustomerID) -- Contraints cannot be declred inside the create in fabric
)

-- Create Dim_Item Table
if not exists (select * from INFORMATION_SCHEMA.TABLES where table_schema = 'sales_schema' and table_name ='Dim_Item')
create table sales_schema.Dim_Item(
    ItemID varchar(255) not null
    ,ItemName varchar(255) not null
)

-- Create Fact_Sales Table 
if not exists (select* from INFORMATION_SCHEMA.TABLES where table_schema = 'sales_schema' and table_name = 'Fact_Sales' )
create table sales_schema.Fact_Sales (
    CustomerID varchar(255) not null
    ,ItemID varchar(255) not null
    ,SalesOrderNumber varchar(30) null
    ,SalesOrderLineNumber int null 
    ,OrderDate date null 
    ,Quantity int null 
    ,TaxAmount float null
    ,UnitPrice float null 
)


In [6]:
%%sql
-- Verifying the creation of our dim and fact tables
SELECT *
FROM [sales_dwh].[INFORMATION_SCHEMA].[TABLES]
where table_schema = 'sales_schema'

<img src="table 2.png" width=800>

In [8]:
%%sql
-- Primary & Foreign Keys creation:

-- Defining primary key for Dim_Customer
alter table sales_schema.Dim_Customer
add constraint PK_Dim_Customer primary key Nonclustered (CustomerID) not enforced

-- Defining primary key for Dim_Item
alter table sales_schema.Dim_Item 
add constraint PK_Dim_Item primary key nonclustered (ItemID) not enforced

-- Defining foreign keys for Fact_Sales
alter table sales_schema.Fact_Sales
add constraint FK_Dim_Item foreign key (ItemID) references sales_schema.Dim_Item (ItemID) not enforced

alter table sales_schema.Fact_Sales
add constraint FK_Dim_Customer foreign key (CustomerID) references sales_schema.Dim_Customer(CustomerID) not enforced

In [9]:
%%sql
-- having a look at the data in the staging table Sales_stg that is located in the data lakehouse 
select top 5 *
from sales_dlh.dbo.sales_stg

<img src="table 3.png" width=800>

3. **<u>Data Loading via Stored Procedure**

In [13]:
%%sql
-- Load data to the warehouse using a stored procedure

create or alter procedure sales_schema.LoadDataFromStaging (@OrderYear int) as 
begin

-- Insert customers data into Dim_Customer while avoiding duplicates.
insert into sales_schema.Dim_Customer (CustomerID, CustomerName,EmailAddress)
select distinct 
    CustomerName
    ,CustomerName
    ,EmailAddress
from sales_dlh.dbo.sales_stg
where year(OrderDate)=@OrderYear
and not exists (
    select * 
    from sales_schema.Dim_Customer c 
    inner join sales_dlh.dbo.sales_stg s
    on  c.CustomerName = s.CustomerName
    and c.EmailAddress = s.EmailAddress   
    )


-- Insert items data into Dim_Item
insert into sales_schema.Dim_Item(ItemID,ItemName)
select distinct Item,Item
from sales_dlh.dbo.sales_stg
where year(OrderDate) = @OrderYear
and not exists (
    select *
    from sales_schema.Dim_Item i
    inner join sales_dlh.dbo.sales_stg s
    on i.ItemName = s.Item
)

-- insert sales data in Fact_Sales
insert into sales_schema.Fact_Sales(CustomerID,ItemID,SalesOrderNumber,SalesOrderLineNumber,OrderDate,Quantity,TaxAmount,UnitPrice)
select distinct
    cast(CustomerName as varchar(255))
    ,cast(Item as varchar(255))
    ,cast(SalesOrderNumber as varchar(30))
    ,cast(SalesOrderLineNumber as int) 
    ,cast(OrderDate as date) 
    ,cast(Quantity as int) 
    ,cast(TaxAmount as float)
    ,cast(UnitPrice as float)
from sales_dlh.dbo.sales_stg
where year(OrderDate) = @OrderYear

end

GO
-- Executing the procedure
exec sales_schema.LoadDataFromStaging 2021

1. **<u>Running Analytical Queries**

In [50]:
%%sql
-- Write a SQL query to retrieve the total sales per customer for the year 2021,
--ensuring the data is grouped by customer and ordered from highest to lowest sales.
select top 5
    c.CustomerName
    ,round(sum(s.Quantity*s.UnitPrice),2) as TotalSales
    
from 
    sales_schema.Dim_Customer c 
    left join sales_schema.Fact_Sales s
    on c.CustomerID = s.CustomerID
where year(s.OrderDate) = 2021
group by c.CustomerName
order by 2 desc


<img src="table 4.png" width=500>

<img src="chart 1.png" width=1500>

In [49]:
%%sql
-- Write a SQL query to retrieve the total sales per product for the year 2021,
-- ensuring the data is grouped by product name and ordered from highest to lowest sales.
select top 5
i.ItemName
,round(sum(s.Quantity * s.UnitPrice),2) as TotalSales

from  sales_schema.Dim_Item i 
left join sales_schema.Fact_Sales s
on i.ItemID = s.ItemID
group by i.ItemName
order by 2 desc


<img src="table 5.png" width=500>

<img src="chart 2.png" width=1500>

In [53]:
%%sql
-- Write a SQL query to identify the top customer in terms of total sales for each product category (Helmet, Bike, Gloves) in the year 2021.
-- Ensure the results include the category, customer name, and total sales, ordered from highest to lowest sales.
 WITH CategorizedSales AS (
 SELECT
     CASE
         WHEN i.ItemName LIKE '%Helmet%' THEN 'Helmet'
         WHEN i.ItemName LIKE '%Bike%' THEN 'Bike'
         WHEN i.ItemName LIKE '%Gloves%' THEN 'Gloves'
         ELSE 'Other'
     END AS Category,
     c.CustomerName,
     s.UnitPrice * s.Quantity AS Sales
 FROM sales_schema.Fact_Sales s
 JOIN sales_schema.Dim_Customer c
 ON s.CustomerID = c.CustomerID
 JOIN sales_schema.Dim_Item i
 ON s.ItemID = i.ItemID
 WHERE YEAR(s.OrderDate) = 2021
 ),
 RankedSales AS (
     SELECT
         Category,
         CustomerName,
         SUM(Sales) AS TotalSales,
         ROW_NUMBER() OVER (PARTITION BY Category ORDER BY SUM(Sales) DESC) AS SalesRank
     FROM CategorizedSales
     WHERE Category IN ('Helmet', 'Bike', 'Gloves')
     GROUP BY Category, CustomerName
 )
 SELECT Category, CustomerName, TotalSales
 FROM RankedSales
 WHERE SalesRank = 1
 ORDER BY TotalSales DESC;

<img src="table 6.png" width=500>

<img src="chart 3.png" width=1500>